In [ ]:
import time
import random
import numpy as np
import torch
import torch.nn.functional as F
import cv2
import ipywidgets as widgets
from IPython.display import display

from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

from jetracer.nvidia_racecar import NvidiaRacecar
from torch2trt import TRTModule

from utils import preprocess


In [ ]:
from xy_dataset import XYDataset
import torchvision.transforms as transforms

TRANSFORM = transforms.Compose([
    transforms.ToPILImage(),  
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

dataset = XYDataset('road_following_F', ['apex'], transform=TRANSFORM)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torchvision import models
import torch.nn as nn
class_names = ['bike', 'railroads', 'road', 'stop', 'trafficlights']
# Road following model
road_model = models.resnet18(pretrained=False)
road_model.fc = nn.Linear(512, 2 * len(dataset.categories))
road_model.load_state_dict(torch.load('road_following_model_2_1.pth'))
road_model = road_model.to(device).eval()

# Collision avoidance model
avoid_model = models.resnet18(pretrained=False)
avoid_model.fc = nn.Linear(512, len(class_names))
avoid_model.load_state_dict(torch.load('model3.pth', map_location=device))
avoid_model = avoid_model.to(device).eval()
print("Loaded model: ", avoid_model)

In [ ]:
import os

bike_path = "/home/jetson/snap/yt-dlp/824/Music/Bicycle bell ring sound effect(HD).wav"
railroad_path = "/home/jetson/snap/yt-dlp/824/Music/Railroad Crossing Signal Bell.wav"
green_path = "/home/jetson/snap/yt-dlp/824/Music/3 2 1 go sound effect.wav"
roadfollowing_path = "/home/jetson/snap/yt-dlp/824/Music/Teriyaki Boyz - Tokyo Drift Instrumental.wav"
stop_path = "/home/jetson/snap/yt-dlp/824/Music/You Better Stop - Meme Sound Effect.wav"
yellow_path = "/home/jetson/snap/yt-dlp/824/Music/Car Skidding Sound Effect.wav"

In [ ]:
camera = CSICamera(width=224, height=224, capture_fps=65)
camera.running = True 

car = NvidiaRacecar()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
display(image_widget)



In [ ]:
import subprocess
bg_process = None
object_present = False
prev_label = None

def control_loop():
    global turning, turn_start
    global avoid_model, road_model
    global bike_path, railroad_path, green_path, roadfollowing_path, stop_path, yellow_path
    
while True:
    frame = camera.value
    image_widget.value = bgr8_to_jpeg(frame)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_tensor = preprocess(rgb).to(device)
    status = True
    song = True
    prev_label = None
    with torch.no_grad():
        output = avoid_model(input_tensor)
        label_idx = output.argmax().item()
        label = class_names[label_idx]
        
        if label != prev_label:
            status = True
        
        if label in ['stop', 'railroads', 'trafficlights']:
            if not object_present:
                object_present = True
                if bg_process:
                    print("Stopping road-following music")
                    bg_process.terminate()
                    bg_process = None   
        
        if label == 'stop':
            print('Object detected stop sign')
            if bg_process:
                bg_process.terminate()
                bg_process.wait()
                bg_process = None

            bg_process = subprocess.Popen(['aplay', '-D', 'plughw:2,0', stop_path],
                                  stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            car.throttle = 0.0
            car.steering = -0.5
        elif label == 'railroads':
            if status:
                if bg_process:
                    bg_process.terminate()   #terminate the last audio sound so that it can play the next
                    bg_process.wait()
                    bg_process = None        #make it reset and play the new audio

                print("Playing RAILROAD audio")
                bg_process = subprocess.Popen(['aplay', '-D', 'plughw:2,0', railroad_path],
                                      stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                print('Object detected railroad sign')
                car.throttle = 0.0
                car.steering = -0.5
                time.sleep(10)
                status = False
            else:
                car.throttle = 0.14
        elif label == 'trafficlights':
            print('Object detected trafficlights')
            traffic_state = random.choice(['green', 'yellow', 'red'])
            print(f"Traffic light is {traffic_state.upper()}")
            if traffic_state == 'red':
                print("RED light: stop 6s")
                bg_process = subprocess.Popen(['aplay', '-D', 'plughw:2,0', stop_path],
                                                  stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                car.throttle = 0.0
                car.steering = 0.0
                time.sleep(6)
                continue
            elif traffic_state == 'yellow':
                print("YELLOW light: slowing down")
                bg_process = subprocess.Popen(['aplay', '-D', 'plughw:2,0', yellow_path],
                                                  stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                car.throttle = 0.135
                time.sleep(4)
                continue
            elif traffic_state == 'green':
                bg_process = subprocess.Popen(['aplay', '-D', 'plughw:2,0', green_path],
                                                  stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                os.system(f'aplay -D plughw:2,0 "{green_path}"')
                car.throttle = 0.15
                time.sleep(4)
                continue
        else:
            with torch.no_grad():
                print('NO Object: Starting Road Following')
                if object_present:
                    print("Object gone — restarting road-following music")
                    object_present = False

                if bg_process is None or bg_process.poll() is not None:
                    print("Playing road-following music")
                    bg_process = subprocess.Popen(['aplay', '-D', 'plughw:2,0', roadfollowing_path],
                                                  stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

                STEERING_GAIN = 0.80
                STEERING_BIAS = -0.05
                #Road Following model
                car.throttle = 0.15
                image = camera.value
                image = preprocess(image)
                output = road_model(image).detach().cpu().numpy().flatten()
                x = float(output[0])
                car.steering = x * STEERING_GAIN + STEERING_BIAS
                
